In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Introduction to Machine Learning with Scikit-Learn

Today's workshop, which is presented by the [KAUST Visualization Core Lab (KVL)](https://corelabs.kaust.edu.sa/visualization/), is the second of two *Introduction to Machine Learning with Scikit-Learn* workshops. These workshops will largely follow Chapter 2 of [*Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow*](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) which walks through the process of developing an end-to-end machine learning project with [Scikit-Learn](https://scikit-learn.org/stable/index.html).

## Today's schedule

* Preparing the Data for Machine Learning Algorithms
* Selecting and Training a Model
* Fine Tuning Your Model

# Prepare the data for machine learning algorithms

"Best practice" is to write functions to automate the process of preparing your data for machine learning. Why?

* Allows you to reproduce these transformations easily on any dataset.
* You will gradually build a library of transformation functions that you can reuse in future projects.
* You can use these functions in a "live" system to transform the new data before feeding it to your algorithms.
* This will make it possible for you to easily experiment with various transformations and see which combination of transformations works best.

First we need to load the training data. The code below loads the training dataset that we created last week using stratified sampling on binned value of `median_income`.

In [ ]:
training_df = pd.read_csv("../data/housing/training.csv", index_col="id")

In [ ]:
training_df.info()

In [ ]:
training_df.head()

## Feature Engineering

We are going to start with some basic feature engineering and data cleaning tasks that we discussed in last week's session but that we didn't actually complete. Feature engineering is one of the most important parts of any machine learning project. Feature engineering is often the most labor intensive part of building a machine learning pipeline and often requires extensive expertise/domain knowledge relevant to the problem at hand.

Recently packages such as [featuretools](https://www.featuretools.com/) have been developed to (partially) automate the process of feature engineering. The success of [deep learning](https://en.wikipedia.org/wiki/Deep_learning) in various domains is in significant part due to the fact that deep learning models are able to automatically engineer features that are most useful for solving certain machine learning tasks. In effect deep learng replaces the expensive to acquire expertise/domain knowledge required to hand-engineer predictive features. The story about [space2vec](https://medium.com/dessa-news/space-2-vec-fd900f5566), a deep learning based supernovae classifier developed by machine learning engineers with no expertise in Astronomy that was able to outperform the machine learning solution developed by NERSC scientists, is a recent example of the power of automated feature engineering. The machine learning pipeline developed by NERSC scientists, called [AUTOSCAN](https://portal.nersc.gov/project/dessn/autoscan/), was a significant improvement over the previous solution which relied on manual classification of supernovae by astronomers. However, in order to achieve such high accuracy, the NERSC solution relied on a dataset of hand-engineered features developed by astronomers with over a century of combined training and expertise in the domain. The deep learning algorithm used by space2vec could be applied directly to the raw image data and did not rely on any hand-engineered features.

In [ ]:
def engineer_features(df):
    """Encapsulate feature engineering in a function so it can be easiyl applied to training and testing datasets."""
    _rooms_per_household = (df.loc[:, "total_rooms"]
                              .div(df.loc[:, "households"]))

    _bedrooms_per_room = (df.loc[:, "total_bedrooms"]
                            .div(df.loc[:, "total_rooms"]))

    _population_per_household = (df.loc[:, "population"]
                                   .div(df.loc[:, "households"]))

    new_attributes = {"rooms_per_household": _rooms_per_household,
                      "bedrooms_per_room": _bedrooms_per_room, 
                      "population_per_household": _population_per_household}
    return df.assign(**new_attributes)



In [ ]:
training_df_with_extra_features = engineer_features(training_df)

In [ ]:
training_df_with_extra_features.head()

## Data Cleaning

In [ ]:
training_df_with_extra_features.describe()

Recall that the target variable `median_house_value` as well as attributes `housing_median_age` and `median_income` are all truncated above some threshold value.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 8))
_ = (training_df_with_extra_features.loc[:, ["housing_median_age", "median_income", "median_house_value"]]
                                    .hist(bins=50, ax=ax))


We need to drop all the observations whose values for at least one of these variables match their respective maximum values. We are also going to encapsulate the logic for dropping observations in a function so that we can reuse the same logic later to drop values from the testing data.

In [ ]:
def _drop_max_values(df, attribute):
    threshold = (df.loc[:, attribute]
                   .max())
    return df.loc[df.loc[:, attribute] < threshold, :]


def clean_dataset(df):
    """
    * Median house values were truncated at 500000 USD. Census block groups with median house values 
      equal to this threshold should be excluded from the analysis.
    * Median income values were truncated at 15 (thousand USD). Census block groups with median income
      values equal to this threshold should be excluded from the analysis.
    * Median housing ages were truncated at 52 years. Census block groups with housing median age 
      values equal to this threshold should be excluded from the analysis.
    
    """
    _df = _drop_max_values(df, "median_house_value")
    _df = _drop_max_values(_df, "median_income")
    _df = _drop_max_values(_df, "housing_median_age")
    return _df


In [ ]:
cleaned_training_df = clean_dataset(training_df_with_extra_features)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 8))
_ = (cleaned_training_df.loc[:, ["housing_median_age", "median_income", "median_house_value"]]
                        .hist(bins=50, ax=ax))


Let’s also separate the attributes/features and the labels/targets. Separating the attributes/features from the labels/targets allows us to more easily apply different sets of transformations to these datasets.

In [ ]:
training_features_df = cleaned_training_df.drop("median_house_value", axis=1)
training_target_df = cleaned_training_df.loc[:, ["median_house_value"]]

Most machine learning algorithms will not work with missing data. There are three options for dealing with missing data.

1. Drop any training samples that are missing values for *any* attribute/feature.
2. Drop any attribute/feature with missing values.
3. Explicitly decide how to fill in the missing values.

We can implement any of the above approaches using built-in functionality of Pandas.

In [ ]:
# option 1
(training_features_df.dropna(subset=["total_bedrooms"])
                     .info())

In [ ]:
# option 2
(training_features_df.drop("total_bedrooms", axis=1)
                     .info())

In [ ]:
# option 3
_median = (training_features_df.loc[:, "total_bedrooms"] # save this value for later so you can prepare the testing features!
                               .median())
(training_features_df.fillna({"total_bedrooms": _median})
                     .info())

However, rather than using Pandas I recommend using the [Scikit-Learn](https://scikit-learn.org/stable/index.html). The Scikit-Learn [`impute`](https://scikit-learn.org/stable/modules/impute.html) module contains a number of different algorithms for filling missing values.  

In [ ]:
from sklearn import impute


simple_imputer = impute.SimpleImputer(strategy="median")

The [`impute.SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) is the first Scikit-Learn Transformer that we have encountered. As such now is a good to to discuss the Scikit-Learn application programming interface (API). The Scikit-Learn API is one of the best designed API's around and has heavily influenced API design choices of other libraries in the Python Data Science and Machine Learning ecosystem, in particular [Dask](https://dask.org/) and [NVIDIA RAPIDS](https://rapids.ai/index.html). Familiarly with the Scikit-Learn API will make it easier for you to get started with these libraries.

The Scikit-Learn API is built around the following key concepts.

* Estimators: Any object that can estimate some parameters based on a dataset is called an estimator (e.g., an `impute.SimpleImputer` is an estimator). The estimation itself is performed by the `fit` method, and it takes only a dataset as a parameter (or two for supervised learning algorithms; the second dataset contains the labels). Any other parameter needed to guide the estimation process is considered a *hyperparameter* (such as the `strategy` parameter in `impute.SimpleImputer`), and it must be set as an instance variable (generally via a constructor parameter).

* Transformers: Some estimators (such as an `impute.SimpleImputer`) can also transform a dataset; these are called transformers. Once again, the API is simple: the transformation is performed by the `transform` method with the dataset to transform as a parameter. It returns the transformed dataset. This transformation generally relies on the learned parameters, as is the case for an imputer. All transformers also have a convenience method called `fit_transform` that is equivalent to calling `fit` and then `transform` (but sometimes `fit_transform` is optimized and runs much faster).

* Predictors: Finally, some estimators, given a dataset, are capable of making predictions; they are called predictors. A predictor has a `predict` method that takes a dataset of new instances and returns a dataset of corresponding predictions. It also has a `score` method that measures the quality of the predictions, given a test set (and the corresponding labels, in the case of supervised learning algorithms).

All of an estimator’s hyperparameters are accessible directly via public instance variables (e.g., `simple_imputer.strategy`), and all the estimator’s learned parameters are accessible via public instance variables with an underscore suffix (e.g., `simple_imputer.statistics_`). Finally, Scikit-Learn provides reasonable default values for most parameters which makes it easy to quickly create a baseline working system.

In [ ]:
simple_imputer.fit(training_features_df)

Since the median only exists for numeric atttributes/features, you will need to drop all of the non-numeric attributes/features from the dataset before fitting `simple_imputer`.

In [ ]:
numeric_features_df = training_features_df.drop("ocean_proximity", axis=1)
simple_imputer.fit(numeric_features_df)

Fitting the `simple_impute` will compute the median values for each attribute/feature in the dataset and store the values for later reuse. 

In [ ]:
simple_imputer.statistics_

In [ ]:
# medians computed using Pandas give same results as above
numeric_features_df.median()

To fill any missing value in the original dataset using the median values computed by calling the `fit` method, we call the `tranform` method.

In [ ]:
imputed_numeric_features_df = simple_imputer.transform(numeric_features_df)

In [ ]:
# Z is numpy array and no longer has any missing values
np.any(imputed_numeric_features_df == np.nan)

There is also a `fit_transform` method which combines the calls to `fit` and `transform` in sequence.

In [ ]:
imputed_numeric_features_df = simple_imputer.fit_transform(numeric_features_df)

In [ ]:
simple_imputer.statistics_

## Handling Text and Categorical Attributes

So far we have only discussed how to handle numeric attributes/features. Our dataset contains on non-numeric attribute/feature, `ocean_proximity` which we have good reason to think is important determinant of housing prices.

In [ ]:
non_numeric_features_df = training_features_df.loc[:, ["ocean_proximity"]]

In [ ]:
non_numeric_features_df.head()

While the above might look like arbitrary text, `ocean_proximity` only takes a limited number of values.

In [ ]:
non_numeric_features_df.value_counts()

Machine learning algorithms almost always work with numbers. The Scikit-Learn [`preprocessing`](https://scikit-learn.org/stable/modules/preprocessing.html) module has several strategies for [encoding non-numeric attributes/features](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features). The simplest strategy is called ordinal encoding and is implemented by the [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder) class.

In [ ]:
from sklearn import preprocessing


ordinal_encoder = preprocessing.OrdinalEncoder()

In [ ]:
Z = ordinal_encoder.fit_transform(non_numeric_features_df)

In [ ]:
Z

In [ ]:
ordinal_encoder.categories_

With this representation machine learning algorithms will assume that two nearby values are more similar than two distant values. This may be fine in some cases, for example cases where the the categories have a natural ordering such as “bad,” “average,” “good,” and “excellent”. 

### Exercise

Can anyone see an issue with using an ordinal encoding for our `ocean_proximity` attribute?

### Answer:

The categories for `ocean_proximity` are not obviously ordered. For example category `0` (`<1H Ocean`) and category `4` (`NEAR OCEAN`) are cleary more similar than to categories `1` and `3`, respectively. Also what about the category `3` (`ISLAND`)?

An alternative encoding strategy that is commonly used with categorical features that have not natural ordering is to create one binary attribute per category. In our case we create one attribute equal to `1` when the category is `<1H OCEAN` (and `0` otherwise), another attribute equal to `1` when the category is `INLAND` (and `0` otherwise), and so on. This is called one-hot encoding, because only one attribute will be equal to 1 (hot), while the others will be 0 (cold). These new attributes are sometimes called dummy attributes. Scikit-Learn provides a [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) class to convert categorical values into one-hot vectors.

In [ ]:
one_hot_encoder = preprocessing.OneHotEncoder()
Z = one_hot_encoder.fit_transform(non_numeric_features_df)

In [ ]:
# transformed features are now a sparse matrix
Z

In [ ]:
# convert sparse matrix to dense numpy array 
Z.toarray()

In [ ]:
one_hot_encoder.categories_

Note that if a categorical attribute has a large number of possible categories, then one-hot encoding will result in a large number of input features. This may slow down training and degrade performance. If this happens, you may want to try replacing the categorical attributes/features with useful numerical attributes/features related to the categories: for example, you could replace the `ocean_proximity` feature with the distance to the ocean. Alternatively, you could replace each category with a learnable, low-dimensional vector called an embedding. This approach is called [feature learning](https://en.wikipedia.org/wiki/Feature_learning) or representation learning and is covered in chapters 13 and 17 of textbook).

## Feature Scaling

Machine learning algorithms typically don’t perform well when the input numerical attributes have very different scales.

In [ ]:
training_features_df.describe()

The simplest approach is to rescale features so that they all reside within the same range (typically between 0 and 1). This approach is implemented in Scikit-Learn by the [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler) class.

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
scaled_numeric_features_df = min_max_scaler.fit_transform(imputed_numeric_features_df)

In [ ]:
min_max_scaler.data_min_ # these values will be reused later to rescale the testing features

In [ ]:
min_max_scaler.data_max_ # these values will be reused later to rescale the testing features

But what happens if an attribute has outliers and you apply min-max scaling?

In [ ]:
_ = training_features_df.plot(kind="box", subplots=True, figsize=(24, 8))
plt.tight_layout()

An alternative approach is to rescale features so that they all have zero mean and unit standard deviation. This approach, which is also called standardization, is particularly useful when attributes/features have outliers and when downstream machine learning algorithms assume that attributes/features have a Gaussian or Normal distribution.

In [ ]:
# we will use this to make sure that all numerical features have the same scale
standard_scaler = preprocessing.StandardScaler()

In [ ]:
scaled_numeric_features_df = standard_scaler.fit_transform(imputed_numeric_features_df)

In [ ]:
standard_scaler.mean_ # these values will be reused later to rescale the testing features

In [ ]:
standard_scaler.scale_ # these values will be reused later to rescale the testing features

As with all the transformations, it is important to fit the scalers to the training data only, not to the full dataset (including the test set). Only then can you use them to transform the training set and the test set (and new data).

## Transformation Pipelines

As you can see creating preprocessing pipelines involves quite a lot of steps and each of the steps needs to be executed in the correct order. Fortunately Scikit-Learn allows you to combine estimators together to create [pipelines](https://scikit-learn.org/stable/modules/compose.html#combining-estimators). We can encapsulate all of the preprocessing logic for our numeric attributes as well as the preprocessing logic for our non-numeric attributes into separate instances of the [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) class.

The `Pipeline` constructor takes a list of name/estimator pairs defining a sequence of steps. All but the last estimator must be transformers (i.e., they must have a `fit_transform` method). The names can be anything you like (as long as they are unique). Later we will see how to access the parameters of pipelines using these names when we discuss hyperparameter tuning.

In [ ]:
from sklearn import pipeline


numerical_pipeline = pipeline.Pipeline(
    steps=[
        ('imputer', impute.SimpleImputer(strategy="median")),
        ('standard_scaler', preprocessing.StandardScaler())
    ],
)

categorical_pipeline = pipeline.Pipeline(
    steps=[
        ("one_hot_encoder", preprocessing.OneHotEncoder())
    ],
)

We can then [combine these pipelines](https://scikit-learn.org/stable/modules/compose.html#columntransformer-for-heterogeneous-data) into a single pipeline using the [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer) class. The constructor requires a list of tuples, where each tuple contains a name, a transformer, and a list of names (or indices) of columns that the transformer should be applied to. 

In [ ]:
from sklearn import compose


numerical_attributes = [
    "longitude",
    "latitude",
    "housing_median_age",
    "total_rooms",
    "total_bedrooms",
    "population",
    "households",
    "median_income",
    "rooms_per_household",
    "bedrooms_per_room",
    "population_per_household",
]

categorical_attributes = [
    "ocean_proximity"
]

preprocessing_pipeline = compose.ColumnTransformer(
    transformers=[
        ("numerical_pipeline", numerical_pipeline, numerical_attributes),
        ("categorical_pipeline", categorical_pipeline, categorical_attributes)
    ],
)

Now we can fit the entire preprocessing pipeline to our training features dataset in one go!

In [ ]:
preprocessed_training_features = preprocessing_pipeline.fit_transform(training_features_df)

In [ ]:
type(preprocessed_training_features)

I often find it useful to create a Pandas `DataFrame` from the `preprocessed_training_features` NumPy `ndarray`.

In [ ]:
categories = list(preprocessing_pipeline.named_transformers_["categorical_pipeline"]
                                        .named_steps["one_hot_encoder"]
                                        .categories_[0])

_columns = numerical_attributes + categories
preprocessed_training_features_df = (pd.DataFrame
                                       .from_records(preprocessed_training_features, columns=_columns))

In [ ]:
preprocessed_training_features_df.head()

Another useful class is [`FeatureUnion`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html#sklearn.pipeline.FeatureUnion). `FeatureUnion` combines several transformer objects into a new transformer that combines their output. A `FeatureUnion` takes a list of transformer objects. During fitting, each of these transformers is fit to the data independently. The transformers are applied in parallel, and the feature matrices they output are concatenated side-by-side into a larger matrix.

Finally, estimators can be displayed with a HTML representation when shown in a Jupyter notebook. Visualizing estimators is particularly useful to diagnose or visualize a `Pipeline` with many estimators. This visualization is activated by setting the display option in [sklearn.set_config](https://scikit-learn.org/stable/modules/generated/sklearn.set_config.html#sklearn.set_config).

In [ ]:
from sklearn import set_config


set_config(display='diagram') 

In [ ]:
preprocessing_pipeline

# Select and Train a Model

At last! You framed the problem, you got the data and explored it, you sampled a training set and a test set, and you wrote transformation pipelines to clean up and prepare your data for machine learning algorithms automatically. You are now ready to select and train a Machine Learning model. You might have been wondering if we were every going to make it to this point! Fact is, most of your time developing machine learning solutions to real-world problems will not be spent training machine learning models: most of *your* time will be spent preparing the data for machine learning algorithms and most of the *computer* time will be spent training the machine learning models.

## Training and Evaluating on the Training Dataset

In [ ]:
from sklearn import linear_model


regressor = linear_model.LinearRegression()
regressor.fit(preprocessed_training_features, training_target_df)

Congrats! You have fit your first machine learning model using Scikit-Learn. Now let's evaluate our model's performance using our chosen metric: root mean squared error (RMSE).

In [ ]:
from sklearn import metrics


predictions = regressor.predict(preprocessed_training_features)
mse = metrics.mean_squared_error(training_target_df, predictions)
rmse = mse**0.5

In [ ]:
rmse # units are USD

Linear regression is often a sensible model to start but often underfits datasets with more complex relationships. Let’s train a [`tree.DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html). This is a powerful model, capable of finding complex nonlinear relationships in the data

In [ ]:
from sklearn import tree


regressor = tree.DecisionTreeRegressor()
regressor.fit(preprocessed_training_features, training_target_df)

In [ ]:
predictions = regressor.predict(preprocessed_training_features)
mse = metrics.mean_squared_error(training_target_df, predictions)
rmse = mse**0.5

In [ ]:
rmse

Wait, what!? No error at all? Could this model really be absolutely perfect? Unfortunately it is much more likely that the model has badly overfit the training data. How can you be sure? As we saw earlier, you don’t want to touch the testing dataset until you are ready to launch a model you are confident about, so you need to use part of the training set for training and part of it for model validation.

## Better Evaluation using Cross Validation

The following code use Scikit-Learn [`model_selection.cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) to randomly split the training set into 10 distinct subsets called folds, then it trains and evaluates our model 10 times, picking a different fold for evaluation every time and training on the other 9 folds. The result is an array containing the 10 evaluation scores.

In [ ]:
from sklearn import model_selection


linear_regression_scores = model_selection.cross_val_score(linear_model.LinearRegression(),
                                                           X=preprocessed_training_features,
                                                           y=training_target_df,
                                                           cv=10,
                                                           scoring="neg_mean_squared_error",
                                                           n_jobs=10)

In [ ]:
def display_rmses(rmses):
    print("RMSE mean:", rmses.mean())
    print("RMSE standard deviation:", rmses.std())


In [ ]:
linear_regression_rmses = np.sqrt(-linear_regression_scores)
display_rmses(linear_regression_rmses)

In [ ]:
_random_state = np.random.RandomState(42)
decision_tree_scores = model_selection.cross_val_score(tree.DecisionTreeRegressor(random_state=_random_state),
                                                       X=preprocessed_training_features,
                                                       y=training_target_df,
                                                       cv=10,
                                                       scoring="neg_mean_squared_error",
                                                       n_jobs=10)

In [ ]:
decision_tree_rmses = np.sqrt(-decision_tree_scores)
display_rmses(decision_tree_rmses)

Now the `DecisionTreeRegressor` doesn’t look nearly as good as it did earlier. In fact, it seems to perform worse than the much simpler `LinearRegression` model. Notice that cross-validation allows you to get not only an estimate of the performance of your model, but also a measure of how precise this estimate is (i.e., its standard deviation).

Let’s try one last model now: the [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). Random forests work by training many decision trees on random subsets of the features, then averaging the predictions made by each of the decision trees to arrive at an overall prediction. Building a model on top of many other models is called [ensemble learning](https://en.wikipedia.org/wiki/Ensemble_learning) and it is often a great approach to improve the predictions of your machine learning pipeline.

In [ ]:
from sklearn import ensemble


_random_state = np.random.RandomState(42)
regressor = ensemble.RandomForestRegressor(random_state=_random_state)
regressor.fit(preprocessed_training_features, training_target_df.iloc[:, 0].ravel())

In [ ]:
predictions = regressor.predict(preprocessed_training_features)
mse = metrics.mean_squared_error(training_target_df, predictions)
rmse = mse**0.5

In [ ]:
rmse

In [ ]:
_random_state = np.random.RandomState(42)
random_forest_scores = model_selection.cross_val_score(ensemble.RandomForestRegressor(random_state=_random_state),
                                                       X=preprocessed_training_features,
                                                       y=training_target_df,
                                                       cv=10,
                                                       scoring="neg_mean_squared_error",
                                                       n_jobs=10)

In [ ]:
random_forest_rmses = np.sqrt(-random_forest_scores)
display_rmses(random_forest_rmses)

A `RandomForestRegressor` look very promising. Note that the score on the training set is still much lower than on the validation sets which indicates that this model is still overfitting the training set. Possible solutions for overfitting are to simplify the model, constrain it (i.e., regularize it), or get a lot more training data.

### Exercise

Before we dive into hyperparameter tuning, you should out a few other models from various categories of machine Learning algorithms: in particular take a look at [Nearest Neighbor](https://scikit-learn.org/stable/modules/neighbors.html) and [Support Vector Machine (SVM)](https://scikit-learn.org/stable/modules/svm.html#regression) regression algorithms. Don't spend too much time tweaking the default hyperparameters. The goal is to shortlist two or three promising models for fine-tuning.

In [ ]:
from sklearn import neighbors

In [ ]:
knn_scores = model_selection.cross_val_score(neighbors.KNeighborsRegressor(),
                                             X=preprocessed_training_features,
                                             y=training_target_df,
                                             cv=10,
                                             scoring="neg_mean_squared_error",
                                             n_jobs=10)

In [ ]:
knn_rmses = np.sqrt(-knn_scores)
display_rmses(knn_rmses)

In [ ]:
from sklearn import svm

In [ ]:
svr_scores = model_selection.cross_val_score(svm.SVR(),
                                             X=preprocessed_training_features,
                                             y=training_target_df,
                                             cv=10,
                                             scoring="neg_mean_squared_error",
                                             n_jobs=10)

In [ ]:
svr_rmses = np.sqrt(-svr_scores)
display_rmses(svr_rmses)

# Fine-tune your models

Most common approach to tuning a model is to manually fiddle with the hyperparameters until you find a great combination of hyperparameter values. Needless to day, this approach to model tuning is *very* tedious and not at all scientific. We can do much better!

## Grid Search

Simplest approach is to use Scikit-Learn’s [`model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). All you need to do is tell it which hyperparameters you want it to experiment with and what values to try out. The `model_selection.GridSearchCV` class will then use cross-validation to evaluate all the possible combinations of hyperparameter values and return the best scoring set of hyperparameters according to your specified metric.

In [ ]:
parameter_grid = [
    {'n_estimators': [10, 100], 'max_features': ["auto", "sqrt", "log2"]}, # 2 * 3 = 6 parameter combinations to try
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 4, 8]}, # 1 * 2 * 3 = 6 parameter combinations to try
]

_random_state = np.random.RandomState(42)
random_forest_regressor = ensemble.RandomForestRegressor(random_state=_random_state)

grid_search_cv = model_selection.GridSearchCV(random_forest_regressor,
                                              parameter_grid,
                                              cv=5,
                                              scoring='neg_mean_squared_error',
                                              return_train_score=True,
                                              n_jobs=5,
                                              verbose=10)

grid_search_cv

In [ ]:
_ = grid_search_cv.fit(preprocessed_training_features, training_target_df)

In [ ]:
# RMSE for the best parameters
(-grid_search_cv.best_score_)**0.5

In [ ]:
grid_search_cv.best_params_

In [ ]:
# best_estimator_ is trained with the values from best_params_
grid_search_cv.best_estimator_

You should save every model you experiment with so that you can come back easily to any model you want. Make sure you save both the hyperparameters and the trained parameters as well as the cross-validation scores and perhaps the actual predictions as well. This will allow you to more easily compare scores across model types and compare the types of errors they make. 

In [ ]:
import joblib
import time

timestamp = time.strftime("%Y%m%d-%H%M%S")
_ = joblib.dump(grid_search_cv, f"../results/models/grid-search-cv-random-forest-regressor-{timestamp}.pkl")

For reference here is how you would reload the trained model from the file.

In [ ]:
reloaded_grid_search_cv = joblib.load(f"../results/models/grid-search-cv-random-forest-regressor-{timestamp}.pkl")

In [ ]:
# compare with grid_search_cv.best_params_
reloaded_grid_search_cv.best_params_

## Randomized Search

The grid search approach is fine when you are exploring relatively few combinations but when the hyperparameter search space is large it is often preferable to use [`model_selection.RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) instead. Instead of trying out all possible combinations, `model_selection.RandomizedSearchCV` evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration. This approach has two main benefits.

* More efficient exploration of the hyperparameter space.
* More control over the computing budget you want to allocate to hyperparameter search.


In [ ]:
from scipy import stats


_param_distributions = {
    "n_estimators": stats.geom(p=0.01),
    "min_samples_split": stats.beta(a=1, b=99),
    "min_samples_leaf": stats.beta(a=1, b=999),
}

_random_state = np.random.RandomState(42)
random_forest_regressor = ensemble.RandomForestRegressor(random_state=_random_state)

randomized_search_cv = model_selection.RandomizedSearchCV(
    random_forest_regressor,
    param_distributions=_param_distributions,
    scoring="neg_mean_squared_error",
    random_state=_random_state,
    n_iter=10,
    cv=5,
    n_jobs=5,
    verbose=10
)

randomized_search_cv

In [ ]:
_ = randomized_search_cv.fit(preprocessed_training_features, training_target_df)

In [ ]:
# RMSE for the best parameters
(-randomized_search_cv.best_score_)**0.5

In [ ]:
randomized_search_cv.best_params_

In [ ]:
_timestamp = time.strftime("%Y%m%d-%H%M%S")
_ = joblib.dump(randomized_search_cv.best_estimator_, f"../results/models/randomized-search-cv-random-forest-regressor-{_timestamp}.pkl")

Grid search and randomized search are the two easiest ways to get started with hyperparameter optimization (HPO) within Scikit-Learn. However, increasingly I finfd myself using [Optuna](https://optuna.org/) for my HPO workloads.

## Analyze the Best Models and Their Errors 

You will often gain good insights on the problem by inspecting the best models. For example, the `ensemble.RandomForestRegressor` can indicate the relative importance of each attribute for making accurate predictions.

In [ ]:
_data = (randomized_search_cv.best_estimator_
                             .feature_importances_)
_index = preprocessed_training_features_df.columns
feature_importances = pd.Series(_data, index=_index)

In [ ]:
feature_importances.sort_values(ascending=False)

It looks like only one of the categories of `ocean_proximity` is useful. Based on this information, I might go back and re-encode `ocean_proximity` to be a binary indicator that takes the value of `1` if the category is either `ISLAND`, `NEAR_BAY`, or `NEAR OCEAN` and `0` if the value is `INLAND`. The would reduce the number of features and speed up computation for some machine learning models.

You should also look at the specific errors that your system makes, then try to understand why it makes them and what could fix the problem (adding extra features or getting rid of uninformative ones, cleaning up outliers, etc.).

In [ ]:
_y_true = (training_target_df.values
                             .ravel())
_y_pred = (randomized_search_cv.best_estimator_
                               .predict(preprocessed_training_features))
_prediction_errors = _y_true - _y_pred # positive prediction error indicates model under-predicts housing prices!
preprocessed_training_features_df["prediction_errors"] = _prediction_errors

### Prediction errors have lots of outliers

If your predictions errors exhibit lots of outliers, then you can inspect which training data samples are the ones for which the model makes poor predictions.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
_ = preprocessed_training_features_df.loc[:, "prediction_errors"].plot(kind="box")

In [ ]:
# census block groups for which model under-predicts housing prices
(preprocessed_training_features_df.sort_values("prediction_errors", ascending=False)
                                  .head())

In [ ]:
# census block groups for which model over-predicts housing prices
(preprocessed_training_features_df.sort_values("prediction_errors", ascending=False)
                                  .tail())

### Exploring the geographical distribution of prediction errors

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
_color = preprocessed_training_features_df.loc[:, "prediction_errors"] / 10000
_cmap = plt.get_cmap("viridis")
_ = preprocessed_training_features_df.plot(kind="scatter", x="longitude", y="latitude", c=_color, cmap=_cmap, ax=ax, alpha=0.4)

### Exploring how prediction errors vary with median income

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
_ = preprocessed_training_features_df.plot(kind="scatter", x="median_income", y="prediction_errors", ax=ax, alpha=0.1)

## Evaluate your system on the test dataset

After tweaking your models for a while, you eventually have a system that performs sufficiently well. Now is the time to evaluate the final model on the test set.

In [ ]:
testing_df = pd.read_csv("../data/housing/testing.csv", index_col="id")

In [ ]:
with_engineered_features_df = engineer_features(testing_df)
cleaned_testing_df = clean_dataset(with_engineered_features_df)
testing_features_df = cleaned_testing_df.drop("median_house_value", axis=1, inplace=False)
testing_target_df = cleaned_testing_df.loc[:, "median_house_value"]
preprocessed_testing_features = preprocessing_pipeline.transform(testing_features_df)

In [ ]:
predictions = randomized_search_cv.best_estimator_.predict(preprocessed_testing_features)
np.sqrt(metrics.mean_squared_error(testing_target_df, predictions))

In some cases, such a point estimate of the generalization error will not be quite enough to convince you to launch: what if it is just marginally better than the model currently in production? You might want to have an idea of how precise this estimate is. 

In [ ]:
# example of computing an estimate of the confidence interval for the test set error
confidence = 0.95
squared_errors = (testing_target_df - predictions)** 2
_interval = (stats.t
                  .interval(confidence,
                            squared_errors.size - 1,
                            loc=squared_errors.mean(),
                            scale=stats.sem(squared_errors)))
np.sqrt(_interval)

If you did a lot of hyperparameter tuning, the performance will usually be slightly worse than what you measured using cross-validation (because your system ends up fine-tuned to perform well on the validation data and will likely not perform as well on unknown datasets). It is not the case in this example, but when this happens you must resist the temptation to tweak the hyperparameters to make the numbers look good on the test set; the improvements would be unlikely to generalize to new data.